<a href="https://colab.research.google.com/github/avrland/dataScienceFromScratch/blob/main/Niusbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #Niusbot
Generating funny news titles, learned from bas of 20k news titles, gathered from polish news websites.

Based on textgenrnn:
https://minimaxir.com/2018/05/text-neural-networks/


In [ ]:
#Montowanie Google Drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip3 install tensorflow==2.3.2
#!pip install -q textgenrnn
!pip3 install git+https://github.com/avrland/textgenrnn.git
from textgenrnn import textgenrnn
from datetime import datetime

  Cloning https://github.com/avrland/textgenrnn.git to /tmp/pip-req-build-y903byib
  Running command git clone -q https://github.com/avrland/textgenrnn.git /tmp/pip-req-build-y903byib
     |████████████████████████████████| 462 kB 5.5 MB/s 
  Created wheel for textgenrnn: filename=textgenrnn-2.0.0-py3-none-any.whl size=1734430 sha256=ee60c6453bfbd380125139706dc561444dcb669e62fa41b49886ea293ddd96de
  Stored in directory: /tmp/pip-ephem-wheel-cache-uy70lf8z/wheels/6c/a8/d5/514646cd30812010e24061dd9370a1b07e64bb6f2636b88943
Successfully built textgenrnn


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Fetching title base
We download base from github there and read first lines, to make sure everything was downloaded properly.

In [ ]:
def lineCounter(filename):
  file = open(filename, "r")
  line_count = 0
  for line in file:
      if line != "\n":
          line_count += 1
  return str(line_count)

!pip install wget
import wget
url = 'https://raw.githubusercontent.com/avrland/polishNewsTitleDatabase/main/titles.txt'
filename = '/content/titles.txt'  
wget.download(url, filename)
with open(filename) as fn:  
  ln = fn.readline()
  lncnt = 0
  while lncnt < 5:
       print("Line {}: {}".format(lncnt, ln.strip()))
       ln = fn.readline()
       lncnt += 1
print("Database size: " + lineCounter(filename))

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=d91152291e50b1ae1261c7924b8a1109540e62f893e409f8b0322fb5260633b3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Line 0: 
Line 1: Polka straciła 36 tys. zł: napastnik wykiwał zarówno ją, jak i bank
Line 2: Chrome 86 na Androida pozwoli zaplanować pobieranie. Można już testować
Line 3: Poczta Polska i cyfrowa rewolucja. Identyfikacja RFID przyspieszy wysyłki
Line 4: GOG GALAXY 2.0 łączy siły z Epic Games Store. Jest wreszcie oficjalna integracja
Database size: 76585


# Model and training parameters
Set the textgenrnn model configuration here: the default parameters here give good results for most workflows. (see the [demo notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)

If you are using an input file where documents are line-delimited, make sure to set `line_delimited` to `True`.

In [ ]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 512,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 3,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 10,   # set higher to train the model for longer
    'gen_epochs': 2,   # generates sample text from model after given number of epochs
    'train_size': 0.9,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.2,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [ ]:
import shutil
file_name = '/content/titles.txt'

#Get current data and time for filename
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'niusbot'  # change to set file name of resulting trained models/texts

textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

76,585 texts collected.
Training new model w/ 3-layer, 512-cell Bidirectional LSTMs
Training on 970,981 word sequences.
Epoch 1/10
948/948 [==============================] - 311s 317ms/step - loss: 5.9102 - val_loss: 5.1767 - lr: 0.0040
Epoch 2/10
948/948 [==============================] - ETA: 0s - loss: 4.8524####################
Temperature: 0.2
####################
koronawirus w polsce. najnowsze dane z sanepidu

koronawirus w polsce. najnowsze dane z sanepidu

koronawirus. ile nowych zakażeń w powiecie. zobacz dane na 19.. 2021

####################
Temperature: 0.5
####################
koronawirus. ile nowych zakażeń w powiecie. zobacz dane na 19.. 2021

koronawirus. ile nowych zakażeń w powiecie - 9.. 2021 w powiecie. zobacz dane na 26.. 2021

koronawirus. ile nowych zakażeń w powiecie. zobacz dane na 19

####################
Temperature: 1.0
####################
korea. leo messiego...

koronawirus. w łodzi do restauracji. trzaskowski jeszcze oferują zgonów w tym tempie podczas 

# Generating titles from cached moded

In [ ]:
textgen.generate(10, temperature=1.1)

 10%|█         | 1/10 [00:01<00:10,  1.19s/it]

bardzo drogie złoto katastrofa. aż 650 testów na koronawirusa. przeciwciała utrzymują się do 13 miesięcy po zakażeniu



 20%|██        | 2/10 [00:01<00:06,  1.15it/s]

w toruniu w polsce. zakażenia sars - - 2



 30%|███       | 3/10 [00:02<00:05,  1.17it/s]

...

oświaty 1 mld zł – inwestycje.



 90%|█████████ | 9/10 [00:07<00:00,  1.53it/s]

andrzej wchodzi w świat biznesu służy



100%|██████████| 10/10 [00:09<00:00,  1.10it/s]

rada odrzuciła wolne szczepionki przeciwko covid - 19 czy konkurencja dla google street view zdjęcia. co wiemy o produkcji nowego świata. na street. kiedy jest mecz polski - sport



# Saving current model to Google Drive

In [ ]:
shutil.copy(model_name + '.hdf5', '/content/drive/MyDrive/')
shutil.copy(model_name + '.json', '/content/drive/MyDrive/')
shutil.copy(model_name + '.json', '/content/drive/MyDrive/')

# Retraining

In [ ]:
import shutil
file_name = '/content/titles.txt'

#Pobierz dzisiejszą datę i godzinę
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'niusbot_' + timeNow   # change to set file name of resulting trained models/texts

#textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=False,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

76,585 texts collected.
Training on 971,095 word sequences.
Epoch 1/4
948/948 [==============================] - 377s 397ms/step - loss: 2.3251 - val_loss: 2.7831 - lr: 0.0040
Epoch 2/4
948/948 [==============================] - ETA: 0s - loss: 1.8193####################
Temperature: 0.2
####################
koronawirus w polsce. najnowsze dane o zakażeniach 15.. 2021

koronawirus w polsce. najnowsze statystyki na 21.. 2021 dla wybranych krajów spoza europy. liczba zgonów i potwierdzo

koronawirus w polsce. nowe przypadki zakażenia i śmierci - 9 maja

####################
Temperature: 0.5
####################
koronawirus w polsce. nowe przypadki zakażenia i śmierci - 10 czerwca

w galerii. jest bardziej

koronawirus w polsce. nowe przypadki zakażenia i śmierci - 19 czerwca

####################
Temperature: 1.0
####################
od poniedziałku rusza program pilotażowy

zakażonych koronawirusem. rodzice w 3 infekcje. uczniowie wracają do szkół. dariusz piontkowski w se będzie mieć



# Generating titles from saved model

In [ ]:
modelPath = "/content/drive/MyDrive/NIUSBOT/titles"
textgen = textgenrnn(weights_path=modelPath + '_weights.hdf5',
                       vocab_path=modelPath + '_vocab.json',
                       config_path=modelPath + '_config.json')
textgen.generate(10, temperature=1.1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
